In [1]:
import pandas as pd 
import os 
import numpy as np 
import sys 
import re
import json 
from helpers import match_city_with_filters, get_city_relevance_score, get_early_stopping_round
import ast

In [2]:
data_dir = "/".join(os.getcwd().split("/")[:-2]) + "/data/conv-trs/"
results_dir = data_dir + "multi-agent/results/"
print(results_dir)

/Users/adithisatish/TUM/Academics/thesis/travel-crs/data/conv-trs/multi-agent/results/


In [3]:
listings_df = pd.read_csv(f"{data_dir}kg-generation/new-kg/data/merged_listing.csv")

In [4]:
samples_df = pd.read_csv(f"{data_dir}multi-agent/sample-data/llama3point2_sample.csv")

In [5]:
def get_early_stopping_information(results_dir, threshold):
    
    improvement = []
    for file in os.listdir(results_dir):
        config_id = (file.split("states_")[1]).split(".json")[0]
        
        with open(f"{results_dir}{file}", "r") as f: 
            data = json.load(f)

        filters = ast.literal_eval(samples_df[samples_df['config_id'] == config_id]['config'].values[0])['filters']

        improvement.append(get_early_stopping_round(
            config_id=config_id,
            state=data,
            filters=filters,
            listings=listings_df,
            threshold=threshold
        ))
    
    return improvement

In [6]:
# test 
mami_gemini = f"{results_dir}prompt_ranking_aggressive/gemini2point5flash/states/"

threshold = 0.6

info = get_early_stopping_information(mami_gemini,threshold)

In [7]:
print(len(info))

45


In [8]:
info

[{'config_id': 'c_p_98_pop_medium_medium',
  'round_init': 0,
  'round_stop': 7,
  'collective_offer': ['Innsbruck',
   'Salzburg',
   'Bern',
   'Ljubljana',
   'Sarajevo',
   'Klagenfurt',
   'Sibiu',
   'Brno',
   'Zilina',
   'Rijeka'],
  'success_score': 1.0,
  'improvement': 0.5000000000000001},
 {'config_id': 'c_p_196_pop_medium_hard',
  'round_init': 0,
  'round_stop': 9,
  'collective_offer': ['Innsbruck',
   'Bern',
   'Bergen',
   'Salzburg',
   'Ljubljana',
   'Stavanger',
   'Valencia',
   'Santander',
   'Sofia',
   'Toulouse'],
  'success_score': 0.75,
  'improvement': 0},
 {'config_id': 'c_p_24_pop_low_medium',
  'round_init': 0,
  'round_stop': 6,
  'collective_offer': ['Ljubljana',
   'Perugia',
   'Sarajevo',
   'Dijon',
   'Plovdiv',
   'Ioannina',
   'Coimbra',
   'Wroclaw',
   'Brno',
   'Dresden'],
  'success_score': 0.6666666666666666,
  'improvement': 1.0},
 {'config_id': 'c_p_77_pop_low_medium',
  'round_init': 0,
  'round_stop': 9,
  'collective_offer': ['Brn

In [10]:
for i in info: 
    if i['round_init'] > 0:
        print(i['config_id'], i['round_init'])

c_p_120_pop_medium_sustainable 1
